In [78]:
import os
os.environ["SPARK_HOME"] = "spark-3.5.1-bin-hadoop3"
os.environ["JAVA_HOME"] = "/usr"

In [79]:
!pip install findspark

In [80]:
import findspark
findspark.init()

In [81]:
!pip3 install pyspark==3.5.1

In [82]:
# Инициализация

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Lab2_6407_Fomin") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .getOrCreate()

## Задание

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: **RDD API**, **Dataset API**, **SQL API**.

In [83]:
prog_path = '../data/programming-languages.csv'
posts_path = '../data/posts_sample.xml'

posts = spark.read.format('xml').options(rowTag='row').load(posts_path)

program = spark.read \
      .option("header", True) \
      .option("inferSchema", True) \
      .option("DateTimeFormat", 'M/d/y H:m') \
      .csv(prog_path)

posts.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [84]:
from pyspark.sql.functions import col

posts.select(col("_CreationDate"), col("_ViewCount"), col("_Tags")).show(5)

+--------------------+----------+--------------------+
|       _CreationDate|_ViewCount|               _Tags|
+--------------------+----------+--------------------+
|2008-08-01 02:42:...|     42817|<c#><floating-poi...|
|2008-08-01 03:08:...|     18214|<html><css><inter...|
|2008-08-01 03:17:...|      NULL|                NULL|
|2008-08-01 04:40:...|    555183|<c#><.net><datetime>|
|2008-08-01 04:55:...|    149445|<c#><datetime><ti...|
+--------------------+----------+--------------------+
only showing top 5 rows



In [85]:
from pyspark.sql.functions import split, udf, from_unixtime, unix_timestamp

strip_udf = udf(lambda string: string[1: -1])
selected_posts = (
    posts.filter(col("_Tags").isNotNull())
        .withColumn("year", from_unixtime(unix_timestamp(col("_CreationDate")), "yyyy").cast("int"))
        .withColumn("views_count", col("_ViewCount"))
        .withColumn("tags", split(strip_udf(col("_Tags")), "><"))
        .select(col("year"), col("views_count"), col("tags"))
)

selected_posts.show(truncate=False)

+----+-----------+------------------------------------------------------+
|year|views_count|tags                                                  |
+----+-----------+------------------------------------------------------+
|2008|42817      |[c#, floating-point, type-conversion, double, decimal]|
|2008|18214      |[html, css, internet-explorer-7]                      |
|2008|555183     |[c#, .net, datetime]                                  |
|2008|149445     |[c#, datetime, time, datediff, relative-time-span]    |
|2008|176405     |[html, browser, timezone, user-agent, timezone-offset]|
|2008|123231     |[.net, math]                                          |
|2010|3650       |[c++, character-encoding]                             |
|2010|617        |[sharepoint, infopath]                                |
|2010|1315       |[iphone, app-store, in-app-purchase]                  |
|2010|973        |[symfony1, schema, doctrine, fixtures]                |
|2010|132        |[java]              

Столбец languages распакуем по примеру
<br/>Из записи: <br/>
|2010| 599| [c#, c++, java]|
<br/>Получилось 3 новых записи:<br/>
|2010| 599| c#  |<br/>
|2010| 599| c++ |<br/>
|2010| 599| java|<br/>

In [86]:
from pyspark.sql.functions import explode
exploded_posts = filtered_posts.select(filtered_posts.year, filtered_posts.views, explode(filtered_posts.languages)).withColumnRenamed("col", "language")
exploded_posts.show()

+----+-----+------------------+
|year|views|          language|
+----+-----+------------------+
|2010| 3650|               c++|
|2010| 3650|character-encoding|
|2010|  617|        sharepoint|
|2010|  617|          infopath|
|2010| 1315|            iphone|
|2010| 1315|         app-store|
|2010| 1315|   in-app-purchase|
|2010|  973|          symfony1|
|2010|  973|            schema|
|2010|  973|          doctrine|
|2010|  973|          fixtures|
|2010|  132|              java|
|2010|  419|visual-studio-2010|
|2010|  419|          stylecop|
|2010|  869|           cakephp|
|2010|  869|       file-upload|
|2010|  869|         swfupload|
|2010| 1303|               git|
|2010| 1303|            cygwin|
|2010| 1303|             putty|
+----+-----+------------------+
only showing top 20 rows



In [87]:
from pyspark.sql.functions import lower

prog = (program.drop("wikipedia_url")).withColumn("name",  lower("name"))
prog.show()

+------------+
|        name|
+------------+
|     a# .net|
|  a# (axiom)|
|  a-0 system|
|          a+|
|         a++|
|        abap|
|         abc|
|   abc algol|
|       abset|
|       absys|
|         acc|
|      accent|
|    ace dasl|
|        acl2|
|     act-iii|
|     action!|
|actionscript|
|         ada|
|     adenine|
|        agda|
+------------+
only showing top 20 rows



In [88]:
exploded_posts = exploded_posts.crossJoin(prog).where("language = name")
counted_posts = exploded_posts.groupBy("year", "language").agg({"views": "sum"})
counted_posts.show()

+----+----------+----------+
|year|  language|sum(views)|
+----+----------+----------+
|2013|    erlang|      2302|
|2017|typescript|     29031|
|2017|       sed|        93|
|2013|javascript|    609571|
|2013|        f#|      4317|
|2012|powershell|     17311|
|2019|       php|      3753|
|2017|   haskell|      4040|
|2013|autohotkey|      4421|
|2019|     xpath|        27|
|2015|    racket|       762|
|2017|        go|      1356|
|2018|     perl6|       109|
|2012|        f#|      1222|
|2018|    python|     99996|
|2017|    prolog|        76|
|2014|       php|    238579|
|2018|       x++|        78|
|2014|       lua|       845|
|2019|   haskell|        85|
+----+----------+----------+
only showing top 20 rows



In [89]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window = Window.partitionBy(counted_posts.year).orderBy(col("sum(views)").desc())
tempDF = counted_posts.withColumn("rank", row_number().over(window))
tempDF = tempDF.withColumnRenamed("sum(views)", "views")
tempDF.show()

+----+------------+-------+----+
|year|    language|  views|rank|
+----+------------+-------+----+
|2010|         php|1189629|   1|
|2010|        java| 563211|   2|
|2010|  javascript| 316131|   3|
|2010| objective-c|  97009|   4|
|2010|        ruby|  76215|   5|
|2010|           c|  66587|   6|
|2010|      python|  60672|   7|
|2010|      matlab|  51865|   8|
|2010| applescript|  32305|   9|
|2010|      delphi|  13065|  10|
|2010|           r|  11087|  11|
|2010|        perl|   6820|  12|
|2010|        bash|   6645|  13|
|2010|     haskell|   5601|  14|
|2010|       xpath|   5122|  15|
|2010|actionscript|   5104|  16|
|2010|         sed|   4573|  17|
|2010|  powershell|   4461|  18|
|2010|       scala|   3330|  19|
|2010|          go|   3171|  20|
+----+------------+-------+----+
only showing top 20 rows



In [90]:
tempDF = tempDF.filter(col("rank") <= 10)
result = tempDF.drop("rank")
result = result.orderBy(col("year").asc(), col("views").desc())
result.show()

+----+-----------+-------+
|year|   language|  views|
+----+-----------+-------+
|2010|        php|1189629|
|2010|       java| 563211|
|2010| javascript| 316131|
|2010|objective-c|  97009|
|2010|       ruby|  76215|
|2010|          c|  66587|
|2010|     python|  60672|
|2010|     matlab|  51865|
|2010|applescript|  32305|
|2010|     delphi|  13065|
|2011| javascript| 809078|
|2011|       java| 389834|
|2011|        php| 246770|
|2011|          c| 238277|
|2011|objective-c| 218934|
|2011|     python| 203180|
|2011|       bash|  60805|
|2011|       ruby|  39223|
|2011|       perl|  28502|
|2011|     matlab|  18816|
+----+-----------+-------+
only showing top 20 rows



In [91]:
directory_path = "result_data.parquet"
result.write.parquet(directory_path)

In [92]:
!ls

example_of_runing_report_script_in_azure.png  README.md
hints.md				      result_data.parquet
__init__.py				      spark-3.5.1-bin-hadoop3
Lab2_6407_Fomin.ipynb


In [93]:
# Удаление

import shutil
shutil.rmtree(directory_path)